In [7]:
# autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

import pathmagic
from tools.project import proj_dir

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import torch
from torch import nn
from net.net_stack import ConcatNetStackSandwitch
from net.pool_net import PoolNet
from net.ddff_net import DDFFNetDummy
from net.auto_encoder_blocks import UNet, UNetEncoder, UNetDecoder
from net.coc_depth_net import CoCDepthEncShareNet

device = "cuda"

batch_size = 2
num_frames = 3

x = torch.rand(batch_size, num_frames, 3, 112, 112, device=device)

#unet = UNet(]).to(device)

net_gen_enc = lambda: UNetEncoder([16, 32, 64])
net_gen_dec = lambda: UNetDecoder([64, 30, 16], [16, 32, 64])

ddff_model = DDFFNetDummy(num_frames, input_dim=64, output_dim=64, use_scoring=False)
model = ConcatNetStackSandwitch(net_gen_enc, net_gen_dec, ddff_model, num_frames).to(device)

#out = model(x)
out = model(x)

#print(unet)
#unet_out = unet(x[:,0])

torch.Size([2, 64, 56, 56]) torch.Size([2, 32, 56, 56])
cat
torch.Size([2, 30, 112, 112]) torch.Size([2, 16, 112, 112])
cat
torch.Size([2, 64, 56, 56]) torch.Size([2, 32, 56, 56])
cat
torch.Size([2, 30, 112, 112]) torch.Size([2, 16, 112, 112])
cat
torch.Size([2, 64, 56, 56]) torch.Size([2, 32, 56, 56])
cat
torch.Size([2, 30, 112, 112]) torch.Size([2, 16, 112, 112])
cat


In [9]:
print(out.shape)

torch.Size([2, 3, 1, 112, 112])


In [10]:
import numpy as np

def unet_pair_gen(enc_sizes, dec_sizes):
    return lambda: UNetEncoder(enc_sizes, ), \
           lambda: UNetDecoder(dec_sizes, enc_sizes)


net2 = ConcatNetStackSandwitch(
    *unet_pair_gen([32, 48], [48, 48]),
    PoolNet(
        in_channels=48,
        out_channels=48,
        enc_sizes=[64, 128, 192], #[48, 64, 128, 192],
        dec_sizes=[192, 128, 96], #[192, 128, 96, 48],
        bn_eps=1e-4,
        dec_pool_layers=False
    ),
    num_frames=3
).to(device)


"""
net2 = ConcatNetStackSandwitch(
    *unet_pair_gen([32], [48]),
    CoCDepthEncShareNet(
        in_channels=32,
        out_channels=48,
        enc_sizes=[48, 64, 128], #[48, 64, 128, 192],
        dec_sizes=[128, 96, 48], #[192, 128, 96, 48],
        bn_eps=1e-4,
        dec_pool_layers=False
    ),
    num_frames=num_frames,
    num_decoder=2
).to(device)
"""


out = net2(x)

torch.Size([2, 48, 112, 112]) torch.Size([2, 32, 112, 112])
cat
torch.Size([2, 48, 112, 112]) torch.Size([2, 32, 112, 112])
cat
torch.Size([2, 48, 112, 112]) torch.Size([2, 32, 112, 112])
cat


In [11]:
print(len(out))
print(out[0].shape)
print(out[1].shape)

2
torch.Size([3, 1, 112, 112])
torch.Size([3, 1, 112, 112])


In [12]:
from tools.tools import module_flat_str

print(module_flat_str(net2))

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace)
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace)
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace)
Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace)
Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_r